In [19]:
import pandas as pd
import joblib

# Load CSV
df = pd.read_csv("data.csv", sep=';')

# Fill NaNs with -1
df.fillna(-1, inplace=True)

# Convert float integers to int
def convert_float_int(val):
    if isinstance(val, float) and val.is_integer():
        return int(val)
    else:
        return val

df = df.map(convert_float_int)



# Keep a copy of the original DataFrame with identifiers
df_original = df.copy()

# Drop identifiers to get features
features = df.drop(['AMF_UE_NGAP_ID', 'ip_source', 'Time'], axis=1)

# Load model
model_data = joblib.load('rf_nas_classifier.pkl')
model = model_data['model']
model_features = model_data['features']

# Ensure feature columns match model training order
# Fill missing columns with 0 and order them correctly
for col in model_features:
    if col not in features.columns:
        features[col] = -1

# Reorder columns
features = features[model_features]

# Predict
predictions = model.predict(features)

# Add predictions to original DataFrame
df_original['prediction'] = predictions

# Filter anomalies
anomaly_rows = df_original[df_original['prediction'] == 1]

# Show anomaly rows with identifiers
#print(anomaly_rows[['AMF_UE_NGAP_ID', 'ip_source', 'Time', 'prediction']])
